<a href="https://colab.research.google.com/github/mouni2317/RAG_Bot/blob/main/RAG_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
%pip install llama-index-readers-file pymupdf
%pip install llama-index-embeddings-huggingface
%pip install llama-index-llms-llama-cpp

In [31]:
# sentence transformers
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en")

In [32]:
!pip install llama-cpp-python

In [33]:
!pip install --upgrade pip

In [34]:
!pip install psycopg2-binary pgvector asyncpg "sqlalchemy[asyncio]" greenlet

In [35]:
!pip install pypdf

In [36]:
!pip install -q transformers einops accelerate langchain bitsandbytes

In [37]:
!pip install llama_index

In [38]:
from llama_index.core import VectorStoreIndex,SimpleDirectoryReader,ServiceContext

In [39]:
!pip install llama-index-llms-huggingface
!pip install llama-index-llms-huggingface-api

from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.core import PromptTemplate

In [40]:
documents=SimpleDirectoryReader("/content/data").load_data()
documents

[Document(id_='a27ab905-a4c3-45e2-b353-e1858ac1cffd', embedding=None, metadata={'page_label': '1', 'file_name': 'Understanding-the-Risks-of-Bonds.pdf', 'file_path': '/content/data/Understanding-the-Risks-of-Bonds.pdf', 'file_type': 'application/pdf', 'file_size': 372189, 'creation_date': '2024-08-11', 'last_modified_date': '2024-08-11'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='Understanding the\nRisks of Bonds\nBronagh Traynor\nInvestment Consultant\n', mimetype='text/plain', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'),
 Document(id_='1f3709af-4c6e-43e8-879e-2f2feab1c2dd', embedding=None, metadata={'page_label': '2', 'file_name': 'Understanding-t

In [41]:
system_prompt="""
You are a Q&A assistant. Your goal is to answer questions as
accurately as possible based on the instructions and context provided.
"""
## Default format supportable by LLama2
query_wrapper_prompt=PromptTemplate("<|USER|>{query_str}<|ASSISTANT|>")

In [42]:
from llama_index.llms.llama_cpp import LlamaCPP

# model_url = "https://huggingface.co/TheBloke/Llama-2-13B-chat-GGML/resolve/main/llama-2-13b-chat.ggmlv3.q4_0.bin"
model_url = "https://huggingface.co/TheBloke/Llama-2-13B-chat-GGUF/resolve/main/llama-2-13b-chat.Q4_0.gguf"

llm = LlamaCPP(
    # You can pass in the URL to a GGML model to download it automatically
    model_url=model_url,
    # optionally, you can set the path to a pre-downloaded model instead of model_url
    model_path=None,
    temperature=0.1,
    max_new_tokens=256,
    # llama2 has a context window of 4096 tokens, but we set it lower to allow for some wiggle room
    context_window=3900,
    # kwargs to pass to __call__()
    generate_kwargs={},
    # kwargs to pass to __init__()
    # set to at least 1 to use GPU
    model_kwargs={"n_gpu_layers": 1},
    verbose=True,
)

llama_model_loader: loaded meta data with 19 key-value pairs and 363 tensors from /tmp/llama_index/models/llama-2-13b-chat.Q4_0.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 5120
llama_model_loader: - kv   4:                          llama.block_count u32              = 40
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 13824
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.attention.head_co

In [43]:
service_context=ServiceContext.from_defaults(
    chunk_size=1024,
    llm=llm,
    embed_model=embed_model
)


<ipython-input-43-f6dd2f399444>:1: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context=ServiceContext.from_defaults(


In [44]:
service_context

ServiceContext(llm_predictor=LLMPredictor(system_prompt=None, query_wrapper_prompt=None, pydantic_program_mode=<PydanticProgramMode.DEFAULT: 'default'>), prompt_helper=PromptHelper(context_window=3900, num_output=256, chunk_overlap_ratio=0.1, chunk_size_limit=None, separator=' '), embed_model=HuggingFaceEmbedding(model_name='BAAI/bge-small-en', embed_batch_size=10, callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x77ff0006d570>, num_workers=None, max_length=512, normalize=True, query_instruction=None, text_instruction=None, cache_folder=None), transformations=[SentenceSplitter(include_metadata=True, include_prev_next_rel=True, callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x77ff0006d570>, id_func=<function default_id_func at 0x7800286e5090>, chunk_size=1024, chunk_overlap=200, separator=' ', paragraph_separator='\n\n\n', secondary_chunking_regex='[^,.;。？！]+[,.;。？！]?')], llama_logger=<llama_index.core.service_context_elements.lla

In [45]:
index=VectorStoreIndex.from_documents(documents,service_context=service_context)

In [46]:
index

In [47]:
query_engine = index.as_query_engine()
response=query_engine.query("What is a Bond")


llama_print_timings:        load time =  326029.69 ms
llama_print_timings:      sample time =       2.04 ms /    44 runs   (    0.05 ms per token, 21526.42 tokens per second)
llama_print_timings: prompt eval time =  545579.58 ms /   843 tokens (  647.19 ms per token,     1.55 tokens per second)
llama_print_timings:        eval time = 1265032.00 ms /    43 runs   (29419.35 ms per token,     0.03 tokens per second)
llama_print_timings:       total time = 1810655.48 ms /   886 tokens


In [48]:
print(response)

 A bond is a loan agreement that is typically issued by governments or companies that includes an indenture which is a legal document that gives terms of the loan including the type and timeliness for repayment.


In [ ]:

response=query_engine.query("What is a Risk")
print(response)

Llama.generate: prefix-match hit


In [51]:
response=query_engine.query("What is a Credit Risk")
print(response)

Llama.generate: prefix-match hit

llama_print_timings:        load time =  326029.69 ms
llama_print_timings:      sample time =       1.24 ms /    24 runs   (    0.05 ms per token, 19417.48 tokens per second)
llama_print_timings: prompt eval time = 1288526.97 ms /  1914 tokens (  673.21 ms per token,     1.49 tokens per second)
llama_print_timings:        eval time =  675631.05 ms /    23 runs   (29375.26 ms per token,     0.03 tokens per second)
llama_print_timings:       total time = 1610884.13 ms /  1937 tokens


 Credit risk is the risk that the borrower may default on the payments expected to the lender.


In [ ]:
index=VectorStoreIndex.from_documents(documents,service_context=service_context)